# Detector general de cascos (Helmet / No-helmet) — Notebook listo para Colab

Este notebook:
- Instala Ultralytics (YOLOv8) y herramientas útiles.
- Intenta descargar un dataset público (Hugging Face). Si falla, te permite subir tu dataset en formato YOLO o COCO.
- Convierte datos si es necesario, crea `data.yaml`.
- Entrena un modelo `yolov8n` (rápido).
- Ejecuta inferencia en imagen y vídeo (o webcam).
- Exporta modelo para despliegue (ONNX opcional).

Ejecuta celdas en orden. Usa GPU para entrenamiento (Colab: *Runtime > Change runtime type > GPU*).

In [ ]:
# Instalar dependencias (ejecutar en Colab)
!pip install -q ultralytics datasets pycocotools fiftyone[desktop] roboflow --quiet
!pip install -q opencv-python-headless matplotlib tqdm seaborn

In [ ]:
import os, shutil, json, random, math, sys
from pathlib import Path
import time
import IPython.display as disp
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

from ultralytics import YOLO

def show_image(path, figsize=(10,6)):
    img = Image.open(path)
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

print('Entorno listo.')

In [ ]:
# Intento descargar dataset de Hugging Face. Si falla, habrá fallback para subir ZIP.
from datasets import load_dataset
dataset_name = "UniqueData/helmet_detection"
data_dir = Path("helmet_dataset")
auto_ok = False

print("Intentando descargar dataset desde Hugging Face:", dataset_name)
try:
    ds = load_dataset(dataset_name)
    print("Dataset cargado. Splits:", list(ds.keys()))
    data_dir.mkdir(exist_ok=True)
    auto_ok = True
except Exception as e:
    print("No se pudo descargar automáticamente el dataset:\n", e)
    print("FALLBACK: sube un ZIP con tu dataset en formato YOLO (images/labels) o COCO usando la celda de subida.")
    auto_ok = False

In [ ]:
# Si la descarga automática funcionó, crear estructura YOLO básica en ./helmet_dataset
if 'ds' in globals() and ds is not None and auto_ok:
    print('Preparando estructura YOLO en ./helmet_dataset ...')
    if data_dir.exists():
        shutil.rmtree(data_dir)
    (data_dir/'images'/'train').mkdir(parents=True, exist_ok=True)
    (data_dir/'images'/'val').mkdir(parents=True, exist_ok=True)
    (data_dir/'labels'/'train').mkdir(parents=True, exist_ok=True)
    (data_dir/'labels'/'val').mkdir(parents=True, exist_ok=True)

    split_names = list(ds.keys())
    print('Splits detectados:', split_names)
    def process_split(split, out_img_dir, out_label_dir, max_images=None):
        n = 0
        for i, item in enumerate(ds[split]):
            try:
                img = item.get('image')
                if img is None:
                    path = item.get('img_path') or item.get('image_path') or item.get('file_name')
                    if path is None:
                        continue
                    img = Image.open(path)
                elif hasattr(img, 'to_numpy'):
                    img = Image.fromarray(img.to_numpy())
            except Exception as e:
                # saltar items problemáticos
                continue

            fname = f"{split}_{i:04d}.jpg"
            img_path = out_img_dir / fname
            img.save(img_path)

            objs = item.get('objects') or item.get('annotations') or item.get('labels') or item.get('bboxes')
            label_lines = []
            if objs:
                for obj in objs:
                    bbox = None
                    if isinstance(obj, dict):
                        bbox = obj.get('bbox')
                    elif isinstance(obj, (list, tuple)) and len(obj) == 4:
                        bbox = obj
                    if bbox and len(bbox) == 4:
                        x_min, y_min, x_max, y_max = bbox
                        w_img, h_img = img.size
                        x_c = (x_min + x_max)/2.0 / w_img
                        y_c = (y_min + y_max)/2.0 / h_img
                        w = (x_max - x_min) / w_img
                        h = (y_max - y_min) / h_img
                        cls = 0
                        label_lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")
            with open(out_label_dir / (fname.replace('.jpg', '.txt')), 'w') as f:
                f.write('\n'.join(label_lines))

            n += 1
            if max_images and n >= max_images:
                break

    if 'train' in ds:
        process_split('train', data_dir/'images'/'train', data_dir/'labels'/'train', max_images=2000)
    else:
        first = split_names[0]
        process_split(first, data_dir/'images'/'train', data_dir/'labels'/'train', max_images=2000)

    yaml_text = f"""train: {str((data_dir/'images'/'train').resolve())}
"

In [ ]:
# Fallback: subir ZIP con dataset en formato YOLO (images/labels) o COCO
try:
    from google.colab import files
    import zipfile
    if not auto_ok:
        print('Sube aquí un ZIP con tu dataset en formato YOLO (images/labels) o COCO.')
        uploaded = files.upload()
        if len(uploaded) == 0:
            print('No se subió nada.')
        else:
            for name in uploaded:
                print('Subido:', name)
                if name.lower().endswith('.zip'):
                    with zipfile.ZipFile(name, 'r') as z:
                        z.extractall('uploaded_dataset')
                    print('Descomprimido en ./uploaded_dataset')
                    if Path('uploaded_dataset/images').exists() and Path('uploaded_dataset/labels').exists():
                        if data_dir.exists():
                            shutil.rmtree(data_dir)
                        shutil.move('uploaded_dataset', str(data_dir))
                        print('Dataset colocado en ./helmet_dataset')
                        yaml_text = f"""train: {str((data_dir/'images'/'train').resolve())}
val: {str((data_dir/'images'/'val').resolve())}
nc: 1
names: ['helmet']
"""
                        with open(data_dir/'data.yaml','w') as f:
                            f.write(yaml_text)
                        print('data.yaml creado en ./helmet_dataset/data.yaml')
                    else:
                        print('No se detectó estructura images/labels en el ZIP. Revisa tu ZIP o organiza a mano.')
except Exception as e:
    print('La celda de subida solo funciona en Colab. Si estás en local, crea/coloca el dataset en ./helmet_dataset con estructura YOLO.')

In [ ]:
if (data_dir/'images').exists():
    print('Contenido de helmet_dataset/images (algunos ejemplos):')
    p = list((data_dir/'images').rglob('*.jpg'))[:10]
    for x in p:
        print('-', x)
    if p:
        show_image(str(p[0]))
else:
    print('No se detectó un dataset en ./helmet_dataset.')
    print('Opciones:')
    print('1) Ejecuta la celda de subida y sube un ZIP con la estructura images/labels.')
    print('2) Sube manualmente imágenes y crea anotaciones YOLO (cada .txt por imagen).')
    print('3) Crea un dataset sintético o usa otro dataset público.')

In [ ]:
# Entrenamiento con YOLOv8 (yolov8n). Ajusta epochs / batch si hace falta.
data_yaml = data_dir/'data.yaml'
if not data_yaml.exists():
    print('No existe data.yaml en', data_dir, '\nAsegúrate de crear/colocar tu dataset en formato YOLO y crear data.yaml.')
else:
    print('Iniciando entrenamiento con YOLOv8 (yolov8n). Revisa que el runtime tenga GPU.')
    model = YOLO('yolov8n.pt')
    results = model.train(data=str(data_yaml), epochs=20, imgsz=640, batch=8, name='helmet_exp')
    print('Entrenamiento finalizado. Resultados en runs/train/helmet_exp')
    ckpt = Path('runs/train/helmet_exp/weights/best.pt')
    if ckpt.exists():
        print('Checkpoint guardado en:', ckpt)
    else:
        print('Carpeta de pesos:', list(Path('runs/train').glob('*')))

In [ ]:
# Inferencia en imagen de prueba (si no hay imágenes, sube una)
ckpt = Path('runs/train/helmet_exp/weights/best.pt')
if not ckpt.exists():
    print('No encuentro checkpoint. Usando yolov8n.pt por defecto para demo.')
    model = YOLO('yolov8n.pt')
else:
    model = YOLO(str(ckpt))

try:
    from google.colab import files
    sample_imgs = list((data_dir/'images').rglob('*.jpg')) if (data_dir/'images').exists() else []
    if len(sample_imgs) == 0:
        print('No hay imágenes de prueba en el dataset. Sube una imagen para inferencia.')
        uploaded = files.upload()
        imgs = []
        for fn in uploaded:
            imgs.append(fn)
    else:
        imgs = [str(sample_imgs[0])]
except Exception:
    # en entorno local
    sample_imgs = list((data_dir/'images').rglob('*.jpg')) if (data_dir/'images').exists() else []
    imgs = [str(sample_imgs[0])] if sample_imgs else []

if len(imgs) == 0:
    print('No hay imágenes para inferencia. Sube o coloca alguna en ./helmet_dataset/images.')
else:
    print('Ejecutando inferencia en:', imgs)
    preds = model.predict(source=imgs, save=True, imgsz=640)
    print('Guardado resultado en runs/detect/')
    out_dir = Path('runs/detect')
    if out_dir.exists():
        runs = sorted(out_dir.iterdir(), key=os.path.getmtime)
        last = runs[-1]
        res_images = list(last.glob('*.jpg'))
        if res_images:
            show_image(str(res_images[0]))
        else:
            print('No se encontraron imágenes con predicciones en', last)
    else:
        print('No se creó runs/detect')

In [ ]:
# Inferencia en vídeo (sube un vídeo). Guarda salida en runs/detect/
try:
    from google.colab import files
    print('Sube un vídeo (o coloca la ruta local si ya lo subiste).')
    uploaded = files.upload()
    video_paths = []
    for fn in uploaded:
        video_paths.append(fn)
    if len(video_paths) == 0:
        print('No se subió vídeo. Termina aquí o sube un archivo y vuelve a ejecutar.')
    else:
        video_in = video_paths[0]
        print('Procesando vídeo:', video_in)
        res = model.predict(source=video_in, save=True, stream=False, imgsz=640)
        print('Resultado guardado en runs/detect/ (revisa la carpeta para el .mp4)')
except Exception as e:
    print('La celda de vídeo funciona en Colab. Si estás en local, coloca el vídeo en la ruta y usa model.predict.')

In [ ]:
# Demo de webcam (funciona en Jupyter local; en Colab puede no funcionar establemente)
import cv2
from IPython.display import display, Image as IPyImage, clear_output

def webcam_demo(model, max_frames=200):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print('Imposible abrir webcam.')
        return
    frame_count = 0
    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        results = model.predict(frame, imgsz=640, conf=0.35, verbose=False)
        img_render = results[0].plot()
        _, encoded = cv2.imencode('.jpg', img_render[:, :, ::-1])
        display(IPyImage(data=encoded.tobytes()))
        clear_output(wait=True)
        frame_count += 1
    cap.release()

print('Webcam demo definido. Descomenta la llamada a webcam_demo(model) para ejecutarlo si tu entorno soporta webcam.')

In [ ]:
# Exportar checkpoint a ONNX (útil para CPU / Raspberry)
ckpt = Path('runs/train/helmet_exp/weights/best.pt')
if ckpt.exists():
    m = YOLO(str(ckpt))
    print('Exportando a ONNX...')
    m.export(format='onnx')
    print('Export completado.')
else:
    print('No se encontró checkpoint para exportar. Entrena primero.')

In [ ]:
# Generar CSV de detecciones a partir de un vídeo (frame, timestamp, class, conf, bbox)
import csv
from datetime import timedelta
import cv2

def detections_to_csv(video_path, out_csv='detections_log.csv'):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 25
    frame_idx = 0
    rows = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model.predict(frame, imgsz=640, conf=0.35, verbose=False)
        for det in results[0].boxes.data.tolist() if getattr(results[0], 'boxes', None) is not None else []:
            x1, y1, x2, y2, score, cls = det
            t_seconds = frame_idx / fps
            ts = str(timedelta(seconds=t_seconds))
            rows.append([frame_idx, ts, int(cls), float(score), float(x1), float(y1), float(x2), float(y2)])
        frame_idx += 1
    cap.release()
    with open(out_csv, 'w', newline='') as f:
        w = csv.writer(f)
        w.writerow(['frame','timestamp','class','score','x1','y1','x2','y2'])
        w.writerows(rows)
    print('CSV generado:', out_csv)

# Uso: detections_to_csv('video.mp4', 'helmet_detections.csv')

## Consejos para mejorar tu detector

- **Aumenta dataset**: más imágenes y diversidad (ángulos, iluminación, tipos de casco).
- **Augmentations**: usa aumentos (flip, hue, blur) para robustez.
- **Finetune más epochs**: 50-100 epochs si tienes datos suficientes.
- **Clases adicionales**: separa "helmet" vs "no_helmet" vs "person" o "bike/moto" para lógica más fina.
- **Tracker**: integra StrongSORT/ByteTrack para contar infractores por individuo.
- **Umbral de confianza**: ajustar `conf` en inferencia para reducir falsos positivos/negativos.

¡Listo! Si quieres que genere una versión más pequeña (solo demo con imágenes sintéticas) o que incluya un dataset público concreto (Kaggle/GitHub) dime cuál y lo adapto.